<a href="https://colab.research.google.com/github/gkckdals3/ML_/blob/main/%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C(1%EC%B0%A8%20%EC%88%98%EC%A0%95).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2

In [17]:
import glob
from google.colab import drive
import pandas as pd
drive.mount('/gdrive', force_remount=True)
#train 타겟값 target에 저장
target_table=pd.read_csv('/gdrive/MyDrive/SyntekaBio/train_output.csv').to_numpy()
target=target_table[:,1:].astype(float)
train_data =(sorted(glob.glob('/gdrive/MyDrive/SyntekaBio/train/PNG/*.png')))


Mounted at /gdrive


In [18]:
#전체 데이터 불러오기/데이터 10분에 1로 축소
for i in range (len(train_data)):
  train_data[i]=cv2.imread(train_data[i])
  train_data[i]=cv2.resize(train_data[i],dsize=(128, 128),interpolation=cv2.INTER_AREA)
  train_data[i]=train_data[i].astype(np.float32)/255
train_scaled=np.array(train_data)
train_scaled.shape

(501, 128, 128, 3)

In [19]:
from tensorflow import keras
from sklearn.model_selection import train_test_split

train_scaled.reshape(-1,128,128,3)

array([[[[1.        , 1.        , 0.84313726],
         [1.        , 1.        , 0.84313726],
         [1.        , 1.        , 0.84313726],
         ...,
         [1.        , 1.        , 0.84313726],
         [1.        , 1.        , 0.84313726],
         [1.        , 0.        , 0.        ]],

        [[1.        , 1.        , 0.84313726],
         [1.        , 0.        , 0.        ],
         [1.        , 1.        , 0.84313726],
         ...,
         [1.        , 1.        , 0.84313726],
         [1.        , 1.        , 0.84313726],
         [1.        , 1.        , 0.84313726]],

        [[1.        , 1.        , 0.84313726],
         [1.        , 1.        , 0.84313726],
         [1.        , 1.        , 0.84313726],
         ...,
         [1.        , 0.        , 0.        ],
         [1.        , 1.        , 0.84313726],
         [1.        , 1.        , 0.84313726]],

        ...,

        [[1.        , 1.        , 0.84313726],
         [1.        , 1.        , 0.84313726]

In [20]:
from sklearn.model_selection import train_test_split

train_scaled,val_scaled,train_target,val_target=train_test_split(train_scaled,target,test_size=0.2)
print('학습 데이터 shape : ', train_scaled.shape)
print('검증 데이터 shape : ', val_scaled.shape)
print('학습 타겟 shape : ', train_target.shape)
print('검증 타겟 shape : ', val_target.shape)

학습 데이터 shape :  (400, 128, 128, 3)
검증 데이터 shape :  (101, 128, 128, 3)
학습 타겟 shape :  (400, 1)
검증 타겟 shape :  (101, 1)


In [21]:
import keras
model = keras.Sequential()
model.add(keras.layers.Conv2D(128, kernel_size=(3,3), activation='relu', kernel_initializer='he_uniform', input_shape=(128,128,3)))
model.add(keras.layers.MaxPooling2D(padding='SAME'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256, activation='relu', kernel_initializer='he_uniform'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [22]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 126, 126, 128)     3584      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 63, 63, 128)       0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 63, 63, 128)       0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 508032)            0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               130056448 
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)              

In [23]:
import tensorflow
from tensorflow.keras.optimizers import Adam
model.compile(loss='binary_crossentropy',
              optimizer = Adam(learning_rate=0.01), 
              metrics=['accuracy'])
print(train_scaled.shape)
print(train_target.shape)

(400, 128, 128, 3)
(400, 1)


In [24]:
history = model.fit(
            train_scaled, train_target, 
            validation_data = (val_scaled,val_target),
            batch_size = 32,
            epochs = 30
                    )

Epoch 1/30
13/13 [==============================] - 12s 841ms/step - loss: 1803.2069 - accuracy: 0.5475 - val_loss: 6.9289 - val_accuracy: 0.3267
Epoch 2/30
13/13 [==============================] - 11s 822ms/step - loss: 19.4084 - accuracy: 0.5500 - val_loss: 5.9894 - val_accuracy: 0.6733
Epoch 3/30
13/13 [==============================] - 11s 824ms/step - loss: 2.4666 - accuracy: 0.5775 - val_loss: 0.6781 - val_accuracy: 0.6733
Epoch 4/30
13/13 [==============================] - 11s 825ms/step - loss: 0.8040 - accuracy: 0.6600 - val_loss: 0.5675 - val_accuracy: 0.6733
Epoch 5/30
13/13 [==============================] - 11s 820ms/step - loss: 0.6277 - accuracy: 0.6650 - val_loss: 0.5241 - val_accuracy: 0.7129
Epoch 6/30
13/13 [==============================] - 11s 819ms/step - loss: 0.5191 - accuracy: 0.7375 - val_loss: 0.4101 - val_accuracy: 0.8317
Epoch 7/30
13/13 [==============================] - 11s 829ms/step - loss: 0.2705 - accuracy: 0.9075 - val_loss: 0.2709 - val_accuracy: 0.